In [21]:
pwd

'c:\\Users\\bhand\\OneDrive\\Desktop\\Deep Learning\\Chicken_disease'

In [22]:
ls

 Volume in drive C is OS
 Volume Serial Number is E641-77AD

 Directory of c:\Users\bhand\OneDrive\Desktop\Deep Learning\Chicken_disease

11-11-2023  20:40    <DIR>          .
30-10-2023  22:20    <DIR>          ..
11-11-2023  18:28    <DIR>          Chicken-Disease-Classification-Tutorial
09-11-2023  00:04    <DIR>          logs
               0 File(s)              0 bytes
               4 Dir(s)  12,021,948,416 bytes free


In [23]:
cd Chicken-Disease-Classification-Tutorial

c:\Users\bhand\OneDrive\Desktop\Deep Learning\Chicken_disease\Chicken-Disease-Classification-Tutorial


In [31]:
import os

In [32]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [33]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [34]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    
    def get_prepare_callbacks_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        
        prepare_callback_config = PrepareCallbackConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )
        
        return prepare_callback_config

In [35]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [36]:
class PrepareCallback:
    def __init__(self, config:PrepareCallbackConfig):
        self.config = config
    
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H:%M:%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [37]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2023-11-11 20:46:02,437: INFO: common: yaml file: C:\Users\bhand\OneDrive\Desktop\Deep Learning\Chicken_disease\Chicken-Disease-Classification-Tutorial\config\config.yaml loaded successfully]
[2023-11-11 20:46:02,440: INFO: common: yaml file: C:\Users\bhand\OneDrive\Desktop\Deep Learning\Chicken_disease\Chicken-Disease-Classification-Tutorial\params.yaml loaded successfully]
[2023-11-11 20:46:02,441: INFO: common: Create directory at: artifacts]
[2023-11-11 20:46:02,444: INFO: common: Create directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-11-11 20:46:02,445: INFO: common: Create directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [38]:
import time
timestamp = time.strftime("%Y-%m-%d-%H:%M:%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2023-11-11-21:02:14'